In [ ]:
import pmdarima as pm
#import auto_arima # requires numpy version 1.26.4 i.e. before 2. 
import time
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_squared_error


In [ ]:
datetime = "2025-03-19_10-47-56"
filename_DD = f'../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv'
filename_FB = f'../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv'
df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=['datetime_hour'])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=['datetime_hour'])

In [ ]:
mycell='871f1b559ffffff'

In [ ]:
df_DD.head(2)

In [ ]:
# # df_DD.groupby("hex_id").agg({'rent_count': 'sum', 
#                                 'return_count': 'sum'})
                            

In [ ]:
# df_DD.rent_count.sum()

In [ ]:
# df_DD.hex_id.unique()

In [ ]:
df_FB.hex_id.unique()

In [ ]:
df_DD.loc[df_DD.hex_id==mycell, ['datetime_hour', 'rent_count', 'return_count']].plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
test_range_1_DD = pd.date_range(start='2024-03-21', end='2024-03-31')
test_range_1_DD = [date.date() for date in test_range_1_DD]

In [ ]:
test_range_2_DD = pd.date_range(start='2024-10-21', end='2024-10-31')
test_range_2_DD = [date.date() for date in test_range_2_DD]


In [ ]:
df_DD_1 = df_DD.loc[df_DD.datetime_hour.dt.date <= test_range_1_DD[-1]]

In [ ]:
df_DD_2 = df_DD.loc[df_DD.datetime_hour.dt.date > test_range_1_DD[-1]]

In [ ]:
train_validation_DD_1 = df_DD_1.loc[~df_DD_1.datetime_hour.dt.date.isin(test_range_1_DD)]

In [ ]:
test_DD_1 = df_DD.loc[df_DD.datetime_hour.dt.date.isin(test_range_1_DD)].sort_values('datetime_hour')

In [ ]:
train_validation_DD_2 = df_DD_2.loc[~df_DD_2.datetime_hour.dt.date.isin(test_range_2_DD)].sort_values('datetime_hour')

In [ ]:
test_DD_2 = df_DD_2.loc[df_DD_2.datetime_hour.dt.date.isin(test_range_2_DD)].sort_values('datetime_hour')

In [ ]:
len(test_DD_2)

In [ ]:
len(train_validation_DD_2)

In [ ]:
train_1 = train_validation_DD_1[train_validation_DD_1.hex_id==mycell].set_index('datetime_hour')['rent_count']

In [ ]:
test_1 = test_DD_1[test_DD_1.hex_id==mycell].set_index('datetime_hour')['rent_count']

In [ ]:
train_1 = train_1.asfreq('h')

In [ ]:
# test_1.asfreq('h').isna().value_counts()

In [ ]:
test_1.asfreq('h').isna().value_counts()

In [ ]:
test_1 = test_1.asfreq('h')

In [ ]:
with open(f"../models/arima DD part 1 {mycell}.pkl", 'rb') as f:
    model = pickle.load(f)  

In [ ]:
model.summary()

In [ ]:
predictions = model.predict(n_periods=len(test_1))
slize_size = 24*5

x = (test_1.index)[:slize_size]
plt.plot(x, test_1.iloc[:slize_size], color='blue')
plt.scatter(x, predictions.iloc[:slize_size], color='yellow')
plt.show()

In [ ]:
mycell

In [ ]:
# MEMO image for the report
plt.figure(figsize=(10, 6)) 
sns.lineplot(data = test_1, label = 'Test data')
sns.lineplot(data = predictions, label='Predictions', linestyle='--')
plt.xlabel('Datetime hour')
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
mean_squared_error(test_1, predictions)